### 分析SelfKG中failure case的分布


In [2]:
# coding: UTF-8
import argparse
import logging
import os
import random
from datetime import datetime
from posixpath import join

import faiss
import numpy as np
import pandas as pd
# using labse
# from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import *

from loader.DBP15KRawNeighbors import DBP15KRawNeighbors
from script.preprocess.deal_raw_dataset import MyRawdataset
from settings import *

In [ ]:
#读取ent_ids转换成id：url的字典
ent_id = "../../Datasets/dbp15k/dbp15k-fr-en-tgt/ent_ids_2"

id2enturl = {}
with open(ent_id, 'r', encoding='utf-8') as f:
    for line in f:
        id_entity = line.strip().split('\t')
        id2enturl[int(id_entity[0])] = id_entity[1].strip()

In [ ]:
#得到head rel tail的url
from os.path import join
from tqdm import *


triple_path = "../../Datasets/dbp15k/dbp15k-fr-en-tgt/triples_2"
path = "../../Datasets/dbp15k/dbp15k-fr-en-tgt/"
name = "rel_triples_whole_2"
h_2_t2rel_url = {}
with open(join(path, name), 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        h_r_t_url = line.strip().split('\t')
        h_url = h_r_t_url[0]
        try:
            r_url = h_r_t_url[1]
        except:
            continue
        t_url = h_r_t_url[2]
        if h_url in id2enturl.values() and t_url in id2enturl.values():
            if not h_url in h_2_t2rel_url.keys():
                h_2_t2rel_url[h_url] = {}
#这个组织形式是head实体包含的所有triple，跟各个tail的关系。是这个head的tuple，是一跳的邻居
            h_2_t2rel_url[h_url][t_url] = r_url

In [ ]:
def fix_seed(seed=37):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.

In [ ]:
len(id_entity)

### 代码组织结构
DBP15KRawNeighbors():  

load读取的是labse编码的entity的embedding    

In [3]:
with open('/home1/data5/bowen/code/SelfKG/data/DBP15K/zh_en/LaBSE_emb_1.pkl', 'rb') as f:
    id_entity = pickle.load(f)

In [23]:
print("the length of id_entity is {}, type is {}".format(len(id_entity),type(id_entity)))
print(len(id_entity[0][0]))
x = np.array(id_entity[0])
x.shape

the length of id_entity is 19388, type is <class 'dict'>
768


(1, 768)

id_neighbors_loader读取的是src/tgt的triple(ids)。这里判断tail是head的邻居.
1. head_str其实是row[head]的embedding  
2. id_neighbors_dict保存的是key:head; value:head的所有tail的embedding
和 key:tail; value:tail的所有head的embedding。保存了所有实体id对应的邻居embedding，第0个代表的是自己。

In [4]:
filepath = '/home1/data5/bowen/code/SelfKG/data/DBP15K/zh_en/triples_1'
data = pd.read_csv(filepath, header=None, sep='\t')
data.columns = ['head', 'relation', 'tail']
id_neighbors_dict = {}
for index, row in data.iterrows():
    head_str = id_entity[int(row['head'])][0]
    tail_str = id_entity[int(row['tail'])][0]

    if not int(row['head']) in id_neighbors_dict.keys():
        id_neighbors_dict[int(row['head'])] = [head_str]
    if not tail_str in id_neighbors_dict[int(row['head'])]:
        id_neighbors_dict[int(row['head'])].append(tail_str)
    
    if not int(row['tail']) in id_neighbors_dict.keys():
        id_neighbors_dict[int(row['tail'])] = [tail_str]
    if not head_str in id_neighbors_dict[int(row['tail'])]:
        id_neighbors_dict[int(row['tail'])].append(head_str)
# len(id_neighbors_dict)

get_adj是创建邻接矩阵，对角，第一行，第一列。每个节点能到自己，第0个表示自己，能到自己的所有邻居，所有邻居能到第0个节点。  

In [5]:
def get_adj(valid_len):
    adj = torch.zeros(NEIGHBOR_SIZE, NEIGHBOR_SIZE).bool()
    for i in range(0, valid_len):
        adj[i, i] = 1
        adj[0, i] = 1
        adj[i, 0] = 1
    return adj

get_center_adj是造出来：
1. id_adj_tensor_dict：dict，是当前实体的邻接矩阵。  
2. id_neighbors_dict:dict, padding扩展到20个neighbors。v是个二维矩阵.
比20大的截断

In [6]:
id_adj_tensor_dict = {}
for k, v in id_neighbors_dict.items():
    if len(v) < NEIGHBOR_SIZE:
        id_adj_tensor_dict[k] = get_adj(len(v))
        id_neighbors_dict[k] = v + [[0]*LaBSE_DIM] * (NEIGHBOR_SIZE - len(v))
    else:
        id_adj_tensor_dict[k] = get_adj(NEIGHBOR_SIZE)
        id_neighbors_dict[k] = v[:NEIGHBOR_SIZE]

In [ ]:
len(v)